In [1]:
import cv2
import numpy as np
import os
import time
from matplotlib import pyplot as plt

Task 1

In [ ]:
path1 = './images'
path2 = './labels'

imgs = os.listdir(path1)
masks = os.listdir(path2)

imgs = list(map(lambda img: cv2.imread(path1 + '/' + img), imgs))
masks = list(map(lambda mask: cv2.imread(path2 + '/' + mask, 0), masks))

res = [cv2.bitwise_and(imgs[i], imgs[i], mask = masks[i]) for i in range(len(imgs))]

In [ ]:
def change_index(key, cur_index = 0):

    if key == ord('1'):
        cur_index += 1

    elif key == ord('2'):
        cur_index -= 1

    else:
        return -1

    if cur_index < 0:
        cur_index = len(res) - 1

    elif cur_index > len(res) - 1:
        cur_index = 0

    return cur_index
    

Task 2

In [ ]:
concat_imgs = [np.concatenate((imgs[i], res[i]), axis = 1) for i in range(len(imgs))]
img_index = 0

while True:
    
    cv2.imshow('Mask', concat_imgs[img_index])

    img_index = change_index(cv2.waitKey(0), img_index)
    if img_index == -1:
        break

cv2.destroyAllWindows()

Task 3

In [ ]:
def make_contour(index):
    
    _, bin = cv2.threshold(masks[index], 0, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(bin, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    return cv2.drawContours(imgs[index], contours, -1, (0, 255, 0), 2)


res = [make_contour(i) for i in range(len(imgs))]
img_index = 0

while True:

    cv2.imshow('Contours', res[img_index])

    img_index = change_index(cv2.waitKey(0), img_index)
    if img_index == -1:
        break

cv2.destroyAllWindows()

Task 4

In [ ]:
cap = cv2.VideoCapture('data/highway_traffic.mp4')

while True:

	ret, frame = cap.read()
	if not ret:
		break
 
	cv2.imshow("Video", cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))

	key = cv2.waitKey(10)
	if key == ord("q"):
		break

cap.release()
cv2.destroyAllWindows()


Laboratory

In [ ]:
tic = time.perf_counter()
is_red = False

cap = cv2.VideoCapture(0)
prev_frame = None

while True:	

    ret, cur_img = cap.read()	
    
    if not ret:
        break
    
    prepared_img = cv2.cvtColor(cur_img, cv2.COLOR_BGR2GRAY)
    prepared_img = cv2.GaussianBlur(prepared_img, ksize = (5, 5), sigmaX = 0)

    if time.perf_counter() - tic >= 10:

        tic = time.perf_counter()
        is_red = not is_red
        prev_frame = None

    if is_red == True:
        
        if prev_frame is None:
            prev_frame = prepared_img
            continue

        diff = cv2.absdiff(prev_frame, prepared_img)
        prev_frame = prepared_img

        kernel = np.ones((5, 5))    
        diff = cv2.dilate(diff, kernel, 1)

        _, thresh_frame = cv2.threshold(diff, 20, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(thresh_frame, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        if not contours:

            _, thresh_frame = cv2.threshold(prepared_img, 50, 255, cv2.THRESH_BINARY)
            contours, _ = cv2.findContours(thresh_frame, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            cv2.drawContours(cur_img, contours, -1, (0, 255, 0), 2)

        else:         
            cv2.drawContours(cur_img, contours, -1, (0, 0, 255), 2)

        cv2.putText(cur_img, 'Red Light', (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    else:
        cv2.putText(cur_img, 'Green Light', (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Frame", cur_img)
    
    key = cv2.waitKey(1)
    if key == ord("q"):
	    break


cap.release()
cv2.destroyAllWindows()